<a href="https://colab.research.google.com/github/armandossrecife/my_validation3/blob/main/my_analysis_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup do Ambiente de Análise

In [1]:
!rm -rf *.log
!rm -rf *.txt
!rm -rf *.xlsx
!rm -rf my_issues
!rm -rf kafka

In [2]:
print('Install Pydriller.')
!pip install pydriller > install_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('Install SQLite in Linux')
!sudo apt install -y sqlite3 > install_sqlite.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install Pydriller.
Install gitpython.
Install Jira Python lib.
Install SQLite in Linux


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
All depenpencies installed!
Details in install.log


## Importa bibliotecas necessárias

In [3]:
import os
from datetime import datetime, timedelta
import tqdm
from pydriller import Repository
from jira import JIRA
import pandas as pd
import re
import scipy.stats as stats
import random
import shutil
import time

## Variáveis Globais

In [15]:
my_repository = 'kafka'
url_to_repository = 'https://github.com/apache/kafka.git'
os.environ['MY_REPOSITORY'] = url_to_repository

JIRA_SERVER = 'https://issues.apache.org/jira'
ISSUE_TRACKER_PROJECT = 'KAFKA'

# Credentials
os.environ['USERNAME'] = 'armandossrecife'
os.environ['PASSWORD'] = 'sky1979#ce'
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

arquivos_kafka_by_sysrepoanalysis = ['StreamThread.java','KafkaConsumer.java','KStream.java','KafkaAdminClient.java','StreamTask.java','Fetcher.java','KafkaStreams.java','TaskManager.java','KStreamImpl.java','KafkaProducer.java','StreamsConfig.java','ConsumerCoordinator.java','Sender.java','AbstractCoordinator.java','KTableImpl.java','KTable.java','StreamsPartitionAssignor.java','NetworkClient.java','InternalTopologyBuilder.java','Protocol.java']

lista_arquivos_criticos = ['StreamThread.java', 'KafkaConsumer.java', 'StreamTask.java', 'Fetcher.java', 'KafkaStreams.java','KStreamImpl.java', 'KafkaProducer.java','StreamsConfig.java', 'ConsumerCoordinator.java']

## Clona o Repositório

In [5]:
print(f'Clona o repositório : {url_to_repository}')
!git clone $MY_REPOSITORY

Clona o repositório : https://github.com/apache/kafka.git
Cloning into 'kafka'...
remote: Enumerating objects: 354010, done.
remote: Counting objects: 100% (1982/1982), done.
remote: Compressing objects: 100% (1207/1207), done.
remote: Total 354010 (delta 621), reused 1365 (delta 341), pack-reused 352028
Receiving objects: 100% (354010/354010), 166.24 MiB | 30.45 MiB/s, done.
Resolving deltas: 100% (170429/170429), done.


In [6]:
!cd kafka && git log --pretty="%H %s" > all_commits_msg.txt
!cd kafka && git log --pretty="%H;%ai;%s" > all_commits_full.txt

# Dados do 1o commit e do ultimo commit
!echo "Último commit: "
!head /content/kafka/all_commits_full.txt -n 1
!echo "Primeiro commit: "
!tail /content/kafka/all_commits_full.txt -n 1

Último commit: 
0390d5b1a24f6f4d0f431928dd009aeb76b3895d;2023-11-02 13:46:05 +0000;KAFKA-15355: Message schema changes (#14290)
Primeiro commit: 
642da2f28c9bc6e373603d6d9119ce33684090f5;2011-08-01 23:41:24 +0000;Initial checkin of Kafka to Apache SVN. This corresponds to https://github.com/kafka-dev/kafka/commit/709afe4ec75489bc00a44335de8821fa726bb97e except that git specific files have been removed and code has been put into trunk/branches/site/etc. This is just a copy of master, branches and history are not being converted since we can't find a good tool for it.


## Funções de apoio

In [7]:
def get_all_commits_by_range(initial_date, final_date, repository_name):
  """Extracts information from a date range of commits
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commit's date, commit's line, commit's file, a list of files modified in the commit, a list of all diffs from modified files
  """
  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
      list_of_modified_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_m_diff = {}
        if m is not None:
          list_of_modified_files.append(m.filename)
          dict_m_diff[m.filename] = m.diff
          list_dict_of_diff_modified_files.append(dict_m_diff)
      data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
      element = commit.msg, data_commit, commit.lines, commit.files, list_of_modified_files, list_dict_of_diff_modified_files
      dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

def get_commits_by_range_and_critical_files(initial_date, final_date, critical_files, repository_name):
  """Extracts information from a date range of commits, focusing on critical files.
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
      critical_files: A list of critical file paths. ex: ['StorageService.java', 'ColumnFamilyStore.java']
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commits's date, commit's line, commit's file, a list of critical files modified in the commit, a list_of_modified_files, a dictionary of diff [filename]:filename.diff, a list of all diffs from modified files
        commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
  """

  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress critical files commit anlysis"):
      list_of_critical_files_modified = []
      list_of_modified_files = []
      list_dict_of_diff_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_diff_file = {}
        list_of_modified_files.append(m.filename)
        list_dict_of_diff_modified_files.append(m.diff)
        if m.filename in critical_files:
          list_of_critical_files_modified.append(m.filename)
          dict_diff_file[m.filename] = m.diff
          list_dict_of_diff_files.append(dict_diff_file)
      if len(list_of_critical_files_modified) > 0:
        data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
        element = commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
        dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

In [8]:
class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments

  def get_comments(self):
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

In [9]:
def analyze_jira_all_issues(project, all_issues):
    """Extracts all issues from Issue Tracker
        Args:
          project: Issue Tracker Project name, example: CASSANDRA
          all_issues: a block of issues
        Returns:
          A list of JiraIssues
    """
    # Create an instance of JiraIssues to manage all issues
    my_all_issues = JiraIssues(project, [])
    total_items = len(all_issues)

    # Iterate through the fetched issues
    for issue in tqdm.tqdm(all_issues, total=total_items, desc='Progress jira all issues analysis'):
        issue_key = issue.key
        issue_summary = issue.fields.summary
        issue_description = issue.fields.description
        issue_comments = [comment.body for comment in issue.fields.comment.comments]

        # Check for SATD keywords in the issue's summary, description, and comments
        if issue_summary is None:
            issue_summary = ""
        if issue_description is None:
            issue_description = ""
        if issue_comments is None:
            issue_comments = ""

        issue_type = issue.fields.issuetype.name
        issue_status = issue.fields.status.name
        issue_priority = issue.fields.priority.name

        # Create a JiraIssue instance for the SATD issue
        my_issue = JiraIssue(issue_key, issue_summary, issue_type, issue_status, issue_priority, issue_description, issue_comments)

        my_all_issues.add_issue(my_issue)

    return my_all_issues

In [10]:
def convert_issues_to_dataframe(all_real_issues):
  l_issue_key_aux, l_issue_type_aux, l_issue_summary_aux, l_issue_description_aux, l_issue_status_aux, l_issue_priority_aux, l_issue_comments_aux = [], [], [], [], [], [], []

  for issue in all_real_issues.get_issues():
    l_issue_key_aux.append(issue.key)
    l_issue_type_aux.append(issue.issue_type)
    l_issue_summary_aux.append(issue.summary)
    l_issue_description_aux.append(issue.description)
    l_issue_status_aux.append(issue.status)
    l_issue_priority_aux.append(issue.priority)
    texto_aux = ""
    for item in issue.get_comments():
      texto_aux = texto_aux + str(item) + "\n"
    l_issue_comments_aux.append(texto_aux)

  dict_all_reall_issues_in_commits_detailed = {
  'issue_key': l_issue_key_aux,
  'issue_type':l_issue_type_aux,
  'status':l_issue_status_aux,
  'priority':l_issue_priority_aux,
  'summary':l_issue_summary_aux,
  'description':l_issue_description_aux,
  'comments':l_issue_comments_aux
  }

  df_all_reall_issues_in_commits_detailed = pd.DataFrame(dict_all_reall_issues_in_commits_detailed)
  return df_all_reall_issues_in_commits_detailed

def convert_commits_to_dataframe(dict_of_commits):
  '''
  v[0] = commit.msg,
  v[1] = data_commit,
  v[2] = commit.lines,
  v[3] = commit.files,
  v[4] = list_of_critical_files_modified,
  v[5] = list_of_modified_files,
  v[6] = list_dict_of_diff_files,
  v[7] = list_dict_of_diff_modified_files
  '''
  l_commit_hash, l_commit_msg, l_commit_data, l_commit_lines, l_commit_files, l_commit_critical_files, l_commit_modified_fies, l_commit_diff_files, l_commit_diff_modified_files = [], [], [], [], [], [], [], [], []
  for k, v in dict_of_commits.items():
    l_commit_hash.append(k)
    l_commit_msg.append(v[0])
    l_commit_data.append(v[1])
    l_commit_lines.append(v[2])
    l_commit_files.append(v[3])
    l_commit_critical_files.append(v[4])
    l_commit_modified_fies.append(v[5])
    l_commit_diff_files.append(v[6])
    l_commit_diff_modified_files.append(v[7])

  dict_of_commits_aux = {
      'hash': l_commit_hash,
      'msg': l_commit_msg,
      'date': l_commit_data,
      'lines': l_commit_lines,
      'files': l_commit_files,
      'critical_files': l_commit_critical_files,
      'modified_files': l_commit_modified_fies,
      'diff_files': l_commit_diff_files,
      'diff_files_modified_files': l_commit_diff_modified_files
  }

  df_commits = pd.DataFrame(dict_of_commits_aux)
  return df_commits

In [11]:
def find_issues_id_by_project(input_string: str, project: str) -> list[str]:
    """Finds all Cassandra issue ID patterns in the input string.
    Args:
        input_string: The input string.
        project: The pattern related to project name, for example: CASSANDRA project name
    Returns:
        A list of project issue IDs, if found; otherwise, an empty list.
    """
    # Try to find all Cassandra issue ID patterns in the input string
    matches = re.findall(r"({0}-\d+)".format(project), input_string)
    # Return an empty list if no matches are found
    if not matches:
        return []
    # Convert the list of matches to a set to remove duplicates
    set_matches = set(matches)
    # Convert the set of matches back to a list
    list_unique_matches = list(set_matches)

    # Return the list of matched Cassandra issue IDs
    return list_unique_matches

def get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files):
  dict_issues_in_commits = {}
  for index in df_commits_with_critical_files.index:
    l_issues_in_commit = find_issues_id_by_project(input_string=df_commits_with_critical_files.msg[index], project=ISSUE_TRACKER_PROJECT)
    if len(l_issues_in_commit) > 0:
      commit_hash = df_commits_with_critical_files.hash[index]
      dict_issues_in_commits[commit_hash] = l_issues_in_commit

  list_issue_commits, list_issue_issues = [], []
  for k, v in dict_issues_in_commits.items():
    list_issue_commits.append(k)
    for issue in v:
      if issue not in list_issue_issues:
        list_issue_issues.append(issue)

  df_aux  = df_all_reall_issues_in_commits_detailed.copy()
  df_issues_in_commits_with_critical_classes = df_aux[df_aux['issue_key'].isin(list_issue_issues)]

  return dict_issues_in_commits, df_issues_in_commits_with_critical_classes

def calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size):
    # Calculate the Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # Calculate the sample size formula
    sample_size = ((z_score**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # Adjust for finite population
    if population_size:
        sample_size = sample_size / (1 + ((sample_size - 1) / population_size))

    return int(sample_size)

In [12]:
def get_max_n_chars(text, max_n):
  text_length = len(text)
  if text_length <= max_n:
    return text
  else:
    return text[:max_n]

def create_new_file(filename, dir_name, issue_type, summary, description, status, comments):
  try:
    filename = dir_name + '/' + filename

    if issue_type is None:
      issue_type = ''
    if summary is None:
      summary = ''
    if description is None:
      description = ''
    if status is None:
      status = ''
    if comments is None:
      comments = ''

    with open(filename, mode='w') as f_issue:
      f_issue.write(f'issue_type: {issue_type} \n')
      f_issue.write(f'summary: {summary} \n')
      f_issue.write(f'description: {get_max_n_chars(text=description, max_n=1000)} \n')
      f_issue.write(f'status: {status} \n')
      f_issue.write(f'comments: {get_max_n_chars(text=comments, max_n=4000)} \n')
    print(f'File {filename} created with success!')

  except Exception as ex:
    print(f'Erro ao criar arquivo: {str(ex)}')


In [13]:
# Seleciona randomicamente os issues para inspeção
def select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023'):
  lista_issues_inspecao = []
  dict_issues_para_inspecao = {}
  list_issue_key = df_issues_in_commits_with_critical_classes.issue_key.to_list()
  list_issue_key = list(set(list_issue_key))
  sample_issues = random.choices(list_issue_key, k=sample_size)
  dict_issues_para_inspecao['02/11/2023'] = sample_issues
  print(f'{len(sample_issues)} para inspeção manual')

  date_file_name = my_date.split('/')
  date_file_name = date_file_name[0] + date_file_name[1] + date_file_name[2]
  file_name = 'issues_inspecao_' + date_file_name + '.txt'
  with open(file_name, mode='w') as f_temp:
    for v in dict_issues_para_inspecao[my_date]:
      elemento = v + ','
      f_temp.write(elemento)
  print(f'Relação de Issues salvos em {my_date} para inspeção.')
  return sample_issues

# Gera os arquivos .txt de cada issue selecionado para inspeção
def generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes):
  contador = 0
  my_dir_name = 'my_issues'
  if not os.path.exists(my_dir_name):
    os.makedirs(my_dir_name)

  total_of_issues = df_issues_in_commits_with_critical_classes.shape[0]
  for index in tqdm.tqdm(df_issues_in_commits_with_critical_classes.index, total=total_of_issues, desc='Analyzing issues'):
    for issue in sample_issues:
      if df_issues_in_commits_with_critical_classes.issue_key[index] == issue:
        create_new_file(filename=df_issues_in_commits_with_critical_classes.issue_key[index], dir_name=my_dir_name, issue_type=df_issues_in_commits_with_critical_classes.issue_type[index], summary=df_issues_in_commits_with_critical_classes.summary[index], description=df_issues_in_commits_with_critical_classes.description[index], status=df_issues_in_commits_with_critical_classes.status[index], comments=df_issues_in_commits_with_critical_classes.comments[index])
        contador += 1
  print(f'Foram criados {contador} arquivos para inspeção')

# 1. Extrai commits de um repositório de código

In [14]:
start_date = datetime(2011, 8, 1, 0, 0, 0)
end_date = datetime(2023, 10, 4, 0, 0, 0)
extract_all_commits = get_all_commits_by_range(initial_date=start_date, final_date=end_date, repository_name=my_repository)

all_commits = extract_all_commits[0]
total_all_commits = extract_all_commits[1]

print(f'Total de commits extraídos: {total_all_commits}')

Wait...


Progress commit anlysis: 100%|██████████| 11732/11732 [03:19<00:00, 58.67it/s]

Total de commits extraídos: 11732


## Filtra os commits com classes críticas

In [16]:
print(f'Analisa a faixa de commits entre: initial_date={str(start_date)}, final_date={str(end_date)}')
print('Registra apenas os commits que contem pelo menos um arquivo crítico')
filter_commits_with_critical_files = get_commits_by_range_and_critical_files(initial_date=start_date, final_date=end_date, critical_files=lista_arquivos_criticos, repository_name=my_repository)
commits_with_critical_files = filter_commits_with_critical_files[0]
total_of_commits_with_critical_files = filter_commits_with_critical_files[1]

print(f'Total de commits com classes críticas: {total_of_commits_with_critical_files}')

Analisa a faixa de commits entre: initial_date=2011-08-01 00:00:00, final_date=2023-10-04 00:00:00
Registra apenas os commits que contem pelo menos um arquivo crítico
Wait...


Progress critical files commit anlysis: 100%|██████████| 11732/11732 [02:07<00:00, 92.09it/s]

Total de commits com classes críticas: 11732


### Converte os commits com classes críticas para dataframe

In [18]:
df_commits_with_critical_files = convert_commits_to_dataframe(dict_of_commits=commits_with_critical_files)
df_commits_with_critical_files

,hash,msg,date,lines,files,critical_files,modified_files,diff_files,diff_files_modified_files
0,269d16d3c915d09f650ae32aa81542bd8522ca68,KAFKA-1227 New producer!,28/1/2014,11720,121,[KafkaProducer.java],"[build.sbt, BufferExhaustedException.java, Cal...","[{'KafkaProducer.java': '@@ -0,0 +1,240 @@ +pa...","[@@ -0,0 +1,11 @@\n+import sbt._\n+import Keys..."
1,253f86e31062fb86401abdc13835c251eef47417,Implement a few of the API suggestions from th...,4/2/2014,1293,38,[KafkaProducer.java],"[Callback.java, DefaultPartitioner.java, Kafka...","[{'KafkaProducer.java': '@@ -6,17 +6,22 @@ imp...","[@@ -2,14 +2,17 @@ package kafka.clients.produ..."
2,fa6339c19cd06880d32ec9a5ee6b66e7f1488dcf,Rename client package from kafka.* to org.apac...,6/2/2014,803,116,[KafkaProducer.java],"[BufferExhaustedException.java, Callback.java,...","[{'KafkaProducer.java': '@@ -1,4 +1,4 @@ -pack...","[@@ -1,6 +1,6 @@\n-package kafka.clients.produ..."
3,36eae8f63c81f8d38c2351054c15bf7d5eb612d9,trivial fix to add missing license header usin...,7/2/2014,2020,125,[KafkaProducer.java],"[HEADER, BufferExhaustedException.java, Callba...","[{'KafkaProducer.java': '@@ -1,3 +1,19 @@ +/**...","[@@ -1,16 +1,14 @@\n-Licensed to the Apache So..."
4,7e154a36f74ad0ea7e0f6d48b71a5a73d99330e7,KAFKA-1261 Make it possible to configure the m...,13/2/2014,67,2,[KafkaProducer.java],"[KafkaProducer.java, ProducerConfig.java]","[{'KafkaProducer.java': '@@ -1,18 +1,14 @@ /*...","[@@ -1,18 +1,14 @@\n /**\n- * Licensed to the ..."
...,...,...,...,...,...,...,...,...,...
1447,9e5ca8416de6011fa8c0d63430e9b89fe751ee2b,MINOR: Fix kafka-site formatting (#14419)\n\nR...,21/9/2023,2,1,[StreamsConfig.java],[StreamsConfig.java],"[{'StreamsConfig.java': '@@ -777,7 +777,7 @@ p...","[@@ -777,7 +777,7 @@ public class StreamsConfi..."
1448,98febb989abd1bdb624420f21122c477f2614a08,"KAFKA-15485: Fix ""this-escape"" compiler warnin...",24/9/2023,159,83,"[KafkaProducer.java, KafkaStreams.java, Stream...","[AbstractCoordinator.java, DefaultBackgroundTh...","[{'KafkaProducer.java': '@@ -333,7 +333,7 @@ p...","[@@ -1291,7 +1291,7 @@ public abstract class A..."
1449,65efb981347d6f81fb2713cd27cdfdfa9d8781b9,KAFKA-10199: Do not process when in PARTITIONS...,26/9/2023,97,2,[StreamThread.java],"[StreamThread.java, StreamThreadTest.java]","[{'StreamThread.java': '@@ -260,6 +260,12 @@ p...","[@@ -260,6 +260,12 @@ public class StreamThrea..."
1450,079e5d647ce39cf2ab5b5f37c5ce28b59fb6db13,KAFKA-15326: [8/N] Move consumer interaction o...,26/9/2023,320,14,"[StreamTask.java, StreamThread.java]","[PartitionGroup.java, ReadOnlyTask.java, Stand...","[{'StreamTask.java': '@@ -84,6 +84,7 @@ public...","[@@ -70,6 +70,7 @@ public class PartitionGroup..."


In [19]:
df_commits_with_critical_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452 entries, 0 to 1451
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   hash                       1452 non-null   object
 1   msg                        1452 non-null   object
 2   date                       1452 non-null   object
 3   lines                      1452 non-null   int64 
 4   files                      1452 non-null   int64 
 5   critical_files             1452 non-null   object
 6   modified_files             1452 non-null   object
 7   diff_files                 1452 non-null   object
 8   diff_files_modified_files  1452 non-null   object
dtypes: int64(2), object(7)
memory usage: 102.2+ KB


In [36]:
df_commits_with_critical_files.to_excel('kafka_commits_with_critical_classes.xlsx', index=False)

# 2. Extrai issues de um issue tracker

## Configura ambiente para acessar Issue Tracker

In [21]:
# Initialize the Jira connection
print('Initialize the Jira connection')
jira = JIRA(JIRA_SERVER, basic_auth=(username, password))

# Create a JiraUtils instance
print('Create a JiraUtils instance')
jira_utils = JiraUtils(ISSUE_TRACKER_PROJECT, jira)

date1 = (2011, 8, 1)
date2 = (2023, 10, 4)
distance = 120
print('Define date intervals')
print(f'From: {str(date1)} to: {date2}, by: {distance} days of distance.')

Initialize the Jira connection
Create a JiraUtils instance
Define date intervals
From: (2011, 8, 1) to: (2023, 10, 4), by: 120 days of distance.


## Recupera os issues do período dado

In [22]:
# Fetch issues using date intervals
print('Fetch issues using date intervals')
block_of_issues = jira_utils.get_list_of_block_issues_by_dates(date1, date2, distance)

Fetch issues using date intervals
Aguarde...


Progress Message Analysis:   3%|▎         | 1/38 [00:02<01:41,  2.75s/it]

Range: project=KAFKA and created>="2011/08/01" and created<="2011/11/28", qtd issues: 135


Progress Message Analysis:   5%|▌         | 2/38 [00:04<01:16,  2.13s/it]

Range: project=KAFKA and created>="2011/11/29" and created<="2012/03/27", qtd issues: 105


...

Range: project=KAFKA and created>="2023/01/30" and created<="2023/05/29", qtd issues: 373


Progress Message Analysis:  97%|█████████▋| 37/38 [02:50<00:04,  4.59s/it]

Range: project=KAFKA and created>="2023/05/30" and created<="2023/09/26", qtd issues: 469


Progress Message Analysis: 100%|██████████| 38/38 [02:53<00:00,  4.56s/it]

Range: project=KAFKA and created>="2023/09/27" and created<="2024/01/24", qtd issues: 265
2023-11-02 16:29:22.028429
Tempo da consulta: 0:02:53.288163


In [28]:
# Concatenate the block of issues into a single list
print('Concatenate the block of issues into a single list')
all_issues = jira_utils.concatenate_block_of_issues(block_of_issues)

Concatenate the block of issues into a single list
Total de issues recuperados: 14309


In [29]:
all_real_issues = analyze_jira_all_issues(ISSUE_TRACKER_PROJECT, all_issues)

Progress jira all issues analysis: 100%|██████████| 14309/14309 [00:00<00:00, 136681.18it/s]


### Converte os issues recuperados para um dataframe

In [30]:
df_all_reall_issues_in_commits_detailed = convert_issues_to_dataframe(all_real_issues)
df_all_reall_issues_in_commits_detailed

,issue_key,issue_type,status,priority,summary,description,comments
0,KAFKA-213,Improvement,Resolved,Major,make # of consumer rebalance retries configurable,,Patch attached.\nA couple of comments -\n\n1. ...
1,KAFKA-212,Bug,Resolved,Major,IllegalThreadStateException in topic watcher f...,If the kafka mirroring embedded consumer recei...,This patch clears the threadList that holds th...
2,KAFKA-211,Task,Resolved,Major,Fix LICENSE file to include MIT and SCALA license,See here for reference - http://markmail.org/s...,+1\nCommitted this.\n
3,KAFKA-210,Improvement,Resolved,Major,javaapi ZookeeperConsumerConnectorTest duplica...,Since javaapi.ZookeeperConsumerConnector is ju...,Remove unnecessary duplicated test cases.\nIs ...
4,KAFKA-209,Improvement,Resolved,Major,Remove empty directory when no log segments re...,When the log cleaner runs it deletes segments ...,"Actually, when all log segments are deleted, w..."
...,...,...,...,...,...,...,...
14304,KAFKA-15514,Sub-task,Resolved,Major,Controller-side replica management changes,"The new ""Assignments"" field replaces the ""Repl...",
14305,KAFKA-15513,Bug,Open,Major,KRaft cluster fails with SCRAM authentication ...,We have observed a scenario where a KRaft clus...,Thank you for the detailed bug report.\r\n\r\n...
14306,KAFKA-15512,Task,Open,Major,EOL zookeeper component in kafka,"Component: *zookeeper 3.6.3* in kafka 3.4.0, ...",Zk has been upgraded to 3.8.2 in upcoming Kafk...
14307,KAFKA-15511,Bug,Resolved,Major,Exception not handled correctly if indexFile ...,I was simulating a code flow where there is ...,"Yes, this is a bug. Would you like to submit a..."


In [39]:
colunas = ['issue_key',	'issue_type', 'status', 'priority', 'summary']
df_all_reall_issues_in_commits_detailed[colunas].to_excel('kafka_all_issues_in_commits.xlsx', index=False)

### Recupera Issues com referência nos commits com classes críticas

In [31]:
dict_issues_in_commits, df_issues_in_commits_with_critical_classes = get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files)
df_issues_in_commits_with_critical_classes

,issue_key,issue_type,status,priority,summary,description,comments
1067,KAFKA-1286,Sub-task,Resolved,Major,Retry Can Block,Under the following scenario the retry logic c...,If you are saying that we repeat the metadata ...
1092,KAFKA-1261,Sub-task,Resolved,Major,Make metadata refresh configurable,,Created reviewboard https://reviews.apache.org...
1093,KAFKA-1260,Sub-task,Closed,Major,Integration Test for New Producer Part II: Bro...,,Created reviewboard https://reviews.apache.org...
1100,KAFKA-1253,Sub-task,Resolved,Major,Implement compression in new producer,,It seems incremental compression would not be ...
1101,KAFKA-1252,Sub-task,Resolved,Major,Implement re-tries in new producer,,Created reviewboard https://reviews.apache.org...
...,...,...,...,...,...,...,...
13594,KAFKA-15485,Improvement,Resolved,Major,Support building with Java 21 (LTS release),JDK-21 is the latest LTS release which reached...,"I started some work for this, so assigned it t..."
13650,KAFKA-15429,Bug,Resolved,Major,Kafka Streams attempts to commit on a closed p...,When a Stream Thread catches an exception from...,Looks like this actually stems from the fix we...
13753,KAFKA-15326,Improvement,Resolved,Critical,Decouple Processing Thread from Polling Thread,As part of an ongoing effort to implement a be...,
13773,KAFKA-15306,Sub-task,Resolved,Major,Integrate committed offsets logic when updatin...,"Integrate refreshCommittedOffsets logic, curre...",merged the PR to trunk\n


In [32]:
df_issues_in_commits_with_critical_classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 939 entries, 1067 to 14026
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   issue_key    939 non-null    object
 1   issue_type   939 non-null    object
 2   status       939 non-null    object
 3   priority     939 non-null    object
 4   summary      939 non-null    object
 5   description  939 non-null    object
 6   comments     939 non-null    object
dtypes: object(7)
memory usage: 58.7+ KB


In [40]:
df_all_reall_issues_in_commits_detailed[colunas].to_excel('kafka_issues_in_commits_with_critical_classes.xlsx', index=False)

# Seleciona amostra dos Issues

In [33]:
confidence_level = 0.95
margin_of_error = 0.05
population_proportion = 0.8
population_size = len(df_issues_in_commits_with_critical_classes)

sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
print(f'População de issues: {population_size}')
print(f"Sample size para inspeção: {sample_size}")

População de issues: 939
Sample size para inspeção: 195


In [34]:
sample_issues = select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023')

195 para inspeção manual
Relação de Issues salvos em 02/11/2023 para inspeção.


In [35]:
generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes)

Analyzing issues:  17%|█▋        | 162/939 [00:00<00:00, 804.92it/s]

File my_issues/KAFKA-1253 created with success!
File my_issues/KAFKA-1250 created with success!
File my_issues/KAFKA-1498 created with success!
File my_issues/KAFKA-2313 created with success!
File my_issues/KAFKA-2123 created with success!
File my_issues/KAFKA-2066 created with success!
File my_issues/KAFKA-2063 created with success!
File my_issues/KAFKA-2763 created with success!
File my_issues/KAFKA-2727 created with success!
File my_issues/KAFKA-2727 created with success!
File my_issues/KAFKA-2723 created with success!
File my_issues/KAFKA-2698 created with success!
File my_issues/KAFKA-2668 created with success!
File my_issues/KAFKA-2668 created with success!
File my_issues/KAFKA-2654 created with success!
File my_issues/KAFKA-2489 created with success!
File my_issues/KAFKA-2459 created with success!
File my_issues/KAFKA-2429 created with success!
File my_issues/KAFKA-2400 created with success!
File my_issues/KAFKA-3245 created with success!
File my_issues/KAFKA-3196 created with s

Analyzing issues:  35%|███▍      | 324/939 [00:00<00:00, 798.57it/s]

File my_issues/KAFKA-3708 created with success!
File my_issues/KAFKA-3703 created with success!
File my_issues/KAFKA-3598 created with success!
File my_issues/KAFKA-3562 created with success!
File my_issues/KAFKA-3562 created with success!
File my_issues/KAFKA-3512 created with success!
File my_issues/KAFKA-3497 created with success!
File my_issues/KAFKA-3439 created with success!
File my_issues/KAFKA-3430 created with success!
File my_issues/KAFKA-3411 created with success!
File my_issues/KAFKA-4163 created with success!
File my_issues/KAFKA-4135 created with success!
File my_issues/KAFKA-4135 created with success!
File my_issues/KAFKA-4033 created with success!
File my_issues/KAFKA-3954 created with success!
File my_issues/KAFKA-4816 created with success!
File my_issues/KAFKA-4724 created with success!
File my_issues/KAFKA-4722 created with success!
File my_issues/KAFKA-4627 created with success!
File my_issues/KAFKA-4561 created with success!
File my_issues/KAFKA-4509 created with s

Analyzing issues:  43%|████▎     | 404/939 [00:00<00:00, 744.55it/s]

...

File my_issues/KAFKA-12699 created with success!
File my_issues/KAFKA-12634 created with success!
File my_issues/KAFKA-12600 created with success!
File my_issues/KAFKA-12568 created with success!
File my_issues/KAFKA-12519 created with success!
File my_issues/KAFKA-12352 created with success!
File my_issues/KAFKA-12313 created with success!
File my_issues/KAFKA-13310 created with success!
File my_issues/KAFKA-13310 created with success!
File my_issues/KAFKA-13217 created with success!
File my_issues/KAFKA-13007 created with success!
File my_issues/KAFKA-12980 created with success!

Analyzing issues:  95%|█████████▍| 890/939 [00:02<00:00, 272.03it/s]


File my_issues/KAFKA-13524 created with success!
File my_issues/KAFKA-13435 created with success!
File my_issues/KAFKA-13846 created with success!
File my_issues/KAFKA-13668 created with success!
File my_issues/KAFKA-14209 created with success!
File my_issues/KAFKA-14379 created with success!


Analyzing issues: 100%|██████████| 939/939 [00:02<00:00, 420.05it/s]

File my_issues/KAFKA-14352 created with success!
File my_issues/KAFKA-14299 created with success!
File my_issues/KAFKA-14279 created with success!
File my_issues/KAFKA-14936 created with success!
File my_issues/KAFKA-14936 created with success!
File my_issues/KAFKA-14936 created with success!
File my_issues/KAFKA-14867 created with success!
File my_issues/KAFKA-14853 created with success!
File my_issues/KAFKA-14834 created with success!
File my_issues/KAFKA-14771 created with success!
File my_issues/KAFKA-14675 created with success!
File my_issues/KAFKA-15429 created with success!
File my_issues/KAFKA-15429 created with success!
Foram criados 195 arquivos para inspeção


In [41]:
!zip -r my_issues.zip my_issues

  adding: my_issues/ (stored 0%)
  adding: my_issues/KAFKA-8348 (deflated 57%)
  adding: my_issues/KAFKA-5755 (deflated 53%)
  adding: my_issues/KAFKA-3245 (deflated 53%)
  adding: my_issues/KAFKA-14675 (deflated 46%)
  adding: my_issues/KAFKA-3562 (deflated 53%)
  adding: my_issues/KAFKA-9298 (deflated 67%)
  adding: my_issues/KAFKA-2313 (deflated 47%)
  adding: my_issues/KAFKA-2489 (deflated 53%)
  adding: my_issues/KAFKA-4857 (deflated 55%)
  adding: my_issues/KAFKA-12247 (deflated 31%)
  adding: my_issues/KAFKA-7367 (deflated 64%)
  adding: my_issues/KAFKA-9821 (deflated 52%)
  adding: my_issues/KAFKA-5052 (deflated 47%)
  adding: my_issues/KAFKA-13217 (deflated 51%)
  adding: my_issues/KAFKA-9753 (deflated 73%)
  adding: my_issues/KAFKA-5597 (deflated 65%)
  adding: my_issues/KAFKA-8285 (deflated 56%)
  adding: my_issues/KAFKA-6687 (deflated 53%)
  adding: my_issues/KAFKA-8319 (deflated 64%)
  adding: my_issues/KAFKA-13435 (deflated 58%)
  adding: my_issues/KAFKA-9581 (deflated 61